In [2]:
import os
import time
import pandas as pd
import numpy as np
from tqdm import tqdm
import itertools

In [ ]:
df = pd.read_csv("Path")

In [ ]:
# Параметры
impulse_range = [0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 5, 6, 7, 8, 9, 10, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 23, 25]
tp_range = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 1.1, 1.2, 1.5, 1.7, 2, 2.5, 3, 3.5, 4, 4.5, 5, 6, 7, 8, 9, 10]
window_range = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 15, 20, 25, 30, 35, 40, 50, 60, 70]
stop_loss_pct = 99  # По сути отключён

results = []

# Подготовка
df["open_time"] = pd.to_datetime(df["open_time"], utc=True)
highs = df["high"].to_numpy(dtype=np.float64)
lows = df["low"].to_numpy(dtype=np.float64)
closes = df["close"].to_numpy(dtype=np.float64)
times = df["open_time"].values.astype("datetime64[ns]")

# Список всех комбинаций
combos = list(itertools.product(impulse_range, tp_range, window_range))
pbar = tqdm(total=len(combos), desc="🔄 Grid Search")

# Перебор
for impulse_threshold, tp, window in combos:
    rolling_max = pd.Series(closes).rolling(window=window).max().to_numpy()
    rolling_min = pd.Series(closes).rolling(window=window).min().to_numpy()
    drawdowns = (rolling_min / rolling_max - 1) * 100
    impulse_mask = drawdowns <= -impulse_threshold

    in_trade = False
    balance = 10
    i = 0
    n = len(closes)
    durations = []
    entry_times = []

    while i < n:
        if not in_trade and impulse_mask[i]:
            entry_price = closes[i]
            entry_time = times[i]
            take_profit = entry_price * (1 + tp / 100)
            stop_loss = entry_price * (1 - stop_loss_pct / 100)
            in_trade = True

            j = i + 1
            while j < n:
                if highs[j] >= take_profit:
                    exit_price = take_profit
                    exit_time = times[j]
                    break
                elif lows[j] <= stop_loss:
                    exit_price = stop_loss
                    exit_time = times[j]
                    break
                j += 1
            else:
                break

            profit_pct = (exit_price / entry_price - 1) * 100
            balance *= (1 + profit_pct / 100)

            duration = (exit_time - entry_time) / np.timedelta64(1, "m")
            durations.append(duration)
            entry_times.append(entry_time)

            in_trade = False
            i = j
        else:
            i += 1

    total_pct = (balance / 10 - 1) * 100

    if durations:
        d_min = np.min(durations)
        d_max = np.max(durations)
        d_mean = np.mean(durations)
        d_median = np.median(durations)
        last_entry = max(entry_times)
    else:
        d_min = d_max = d_mean = d_median = 0
        last_entry = pd.NaT

    results.append({
        "impulse_threshold": impulse_threshold,
        "take_profit": tp,
        "window": window,
        "profit_total_pct": total_pct,
        "duration_min": d_min,
        "duration_max": d_max,
        "duration_mean": d_mean,
        "duration_median": d_median,
        "last_entry_time": last_entry
    })

    pbar.update(1)

pbar.close()
grid_df = pd.DataFrame(results)
grid_df = grid_df.sort_values("profit_total_pct", ascending=False)

# ТОП-10
print("🏆 ТОП-10 лучших комбинаций:")
grid_df.head(10)


In [ ]:
# === СОХРАНЕНИЕ CSV ===
symbol = " "         # <-- замени на актуальную монету
interval = " "            # <-- и таймфрейм (1m, 5m, 1h и т.д.)
combo_count = len(combos)

filename = f"optimization_{symbol}_{interval}_{combo_count}_combos.csv"
grid_df.to_csv(filename, index=False)

print(f"💾 Результаты сохранены: {filename}")
